In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import pathlib

data_root = pathlib.Path.home().joinpath('Documents/Unimelb/Dissertation/test_Data/train_img/')
for item in data_root.iterdir():
  print(item)
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
all_image_labels =[]
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:5])
import numpy as np
# np.save('all_image_paths.npy', all_image_paths)
all_image_paths = np.asarray(all_image_paths)

# One hot vector representation of labels
all_image_labels = np.asarray(all_image_labels)
from keras.utils import to_categorical, Sequence
all_image_labels = to_categorical(all_image_labels, num_classes=3)

# saving the y_labels_one_hot array as a .npy file
# np.save('all_image_labels.npy', all_image_labels)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=6)

from skimage.io import imread
from skimage.transform import resize
import keras

class DataGenerator(keras.utils.Sequence) :
    def __init__(self, image_filenames, labels,
                 batch_size, img_size=(224,224), *args, **kwargs):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        
    def __len__(self):
        return (np.ceil(len(self.image_filenames)/
                        float(self.batch_size))).astype(np.int)
    
    
    def __data_generation(self, file_paths, labels):
        X = np.empty((self.batch_size, *self.img_size, 1))
        Y = np.empty((self.batch_size, 3), dtype=np.float32)
        for i, ID in enumerate(file_paths):
            X[i,] = imread(ID)
        for i, _ in enumerate(labels):
            Y[i,] = labels[i]
        return X, Y
    
    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx*self.batch_size:
                                       (idx+1)*self.batch_size]
        batch_y = self.labels[idx*self.batch_size:(idx+1)*self.batch_size]
        
        x, y = self.__data_generation(batch_x, batch_y)
#         np.array([resize(imread(file_name), (224, 224, 3))
#                for file_name in batch_x])/255.0, np.array(batch_y)

        return x, y


batch_size = 32

training_batch_generator = DataGenerator(x_train, y_train, batch_size)
validation_batch_generator = DataGenerator(x_val, y_val, batch_size)  


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3

# base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
# base_model = VGG19(include_top=False, input_shape=(224, 224, 3))
# base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
# base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
    
    
FC_NUMS = 1024
IMAGE_SIZE = 224
FREEZE_LAYERS = 160
NUM_CLASSES = 3

from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_NUMS, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

from tensorflow.keras.models import Model

# 构造完新的FC层，加入custom层
model = Model(inputs=base_model.input, outputs=prediction)
# 可观察模型结构
# model.summary()
# 获取模型的层数
print("layer nums:", len(model.layers))

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True


from tensorflow.keras.optimizers import SGD, Adam

learing_rate = 1e-3
model.compile(optimizer=SGD(lr=learing_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 20
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))
    
callbacks = [
# 当验证集上的损失“val_loss”连续两个训练回合（epoch）都没有变化，则提前结束训练
tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_loss'),
# 使用TensorBoard保存训练的记录，保存到“./logs”目录中
tf.keras.callbacks.TensorBoard(log_dir='./logs'),
tf.keras.callbacks.LearningRateScheduler(scheduler)]

history = model.fit(training_batch_generator,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs, verbose=1,
                    validation_data = validation_batch_generator,
                   callbacks=callbacks)

/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/.DS_Store
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/down_up
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/down
First 10 labels indices:  [0, 2, 1, 0, 1]
layer nums: 178


ValueError: Failed to find data adapter that can handle input: <class '__main__.DataGenerator'>, <class 'NoneType'>